# 2-guruh: Ehtimollik va taqsimotlar bo'yicha amaliy mashqlar

Bu mashq daftarida siz ehtimollik taqsimotlari bilan chuqurroq ishlaysiz. Murakkab masalalar, real ma'lumotlar bilan ishlash va taqsimotlar kombinatsiyasi o'rganiladi.

**Maqsad:** Taqsimotlar bilan professional darajada ishlash ko'nikmalarini rivojlantirish va real loyihalarda qo'llash.

---

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## Mashq 1: Normal taqsimot - Chuqur tahlil

**Vazifa:** $\mu=10$, $\sigma=3$ bo'lgan normal taqsimotdan 2000 ta son generatsiya qiling va quyidagilarni bajaring:

1. Turli parametrlar bilan normal taqsimotlarni taqqoslang
2. Z-score orqali standartlashtiring
3. Confidence interval (ishonch oralig'i) hisoblang
4. Outlier (chetga chiquvchi qiymatlar)ni aniqlang va tahlil qiling

**Yechim:**

In [ ]:
# Normal taqsimot parametrlari
mu, sigma = 10, 3
data = np.random.normal(mu, sigma, 2000)

# Turli parametrlar bilan taqqoslash
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Turli o'rtacha qiymatlar
x = np.linspace(-5, 25, 1000)
axes[0,0].plot(x, stats.norm.pdf(x, 5, 3), label='μ=5, σ=3', linewidth=2)
axes[0,0].plot(x, stats.norm.pdf(x, 10, 3), label='μ=10, σ=3', linewidth=2)
axes[0,0].plot(x, stats.norm.pdf(x, 15, 3), label='μ=15, σ=3', linewidth=2)
axes[0,0].set_title('Turli o\'rtacha qiymatlar ta\'siri')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Turli standart og'ishlar
axes[0,1].plot(x, stats.norm.pdf(x, 10, 1), label='μ=10, σ=1', linewidth=2)
axes[0,1].plot(x, stats.norm.pdf(x, 10, 3), label='μ=10, σ=3', linewidth=2)
axes[0,1].plot(x, stats.norm.pdf(x, 10, 5), label='μ=10, σ=5', linewidth=2)
axes[0,1].set_title('Turli standart og\'ish ta\'siri')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Z-score standartlashtirish
z_scores = (data - mu) / sigma
axes[1,0].hist(z_scores, bins=50, density=True, alpha=0.7, color='lightblue', edgecolor='black')
z_x = np.linspace(-4, 4, 100)
axes[1,0].plot(z_x, stats.norm.pdf(z_x, 0, 1), 'r-', linewidth=2, label='Standart normal N(0,1)')
axes[1,0].set_title('Z-score standartlashtirilgan ma\'lumotlar')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. Q-Q plot normallik tekshirish uchun
from scipy.stats import probplot
probplot(data, dist="norm", plot=axes[1,1])
axes[1,1].set_title('Q-Q Plot (Normallik tekshiruvi)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Confidence intervals
confidence_levels = [0.90, 0.95, 0.99]
sample_mean = np.mean(data)
sample_std = np.std(data, ddof=1)
n = len(data)

print("Confidence Intervals (Ishonch oraliqlari):")
for conf in confidence_levels:
    alpha = 1 - conf
    t_critical = stats.t.ppf(1 - alpha/2, df=n-1)
    margin_error = t_critical * (sample_std / np.sqrt(n))
    ci_lower = sample_mean - margin_error
    ci_upper = sample_mean + margin_error
    print(f"{conf*100}% CI: [{ci_lower:.3f}, {ci_upper:.3f}]")

# Outlier detection (IQR method va Z-score method)
Q1 = np.percentile(data, 25)
Q3 = np.percentile(data, 75)
IQR = Q3 - Q1
outliers_iqr = data[(data < Q1 - 1.5*IQR) | (data > Q3 + 1.5*IQR)]

# Z-score method (|z| > 3)
outliers_zscore = data[np.abs(z_scores) > 3]

print(f"\nOutlier Detection:")
print(f"IQR usuli bilan topilgan outlierlar soni: {len(outliers_iqr)}")
print(f"Z-score usuli bilan topilgan outlierlar soni: {len(outliers_zscore)}")
print(f"Umumiy ma'lumotlar soni: {len(data)}")
print(f"Outlierlar foizi (IQR): {len(outliers_iqr)/len(data)*100:.2f}%")

## Mashq 2: Binomial taqsimot - Real loyiha simulatsiyasi

**Vazifa:** Internetda reklama kliklari uchun A/B test simulatsiyasi:
- Variant A: $n=15$, $p=0.3$ (30% klik darajasi)
- Variant B: $n=15$, $p=0.4$ (40% klik darajasi)

Quyidagilarni bajaring:
1. Har ikkala variant uchun taqsimotlarni chizing
2. 1000 marta eksperiment simulatsiyasini o'tkazing
3. Qaysi variant yaxshiroq ekanligini statistik jihatdan tahlil qiling
4. Normal yaqinlashuvni tekshiring

**Yechim:**

In [ ]:
# A/B Test parametrlari
n = 15  # Number of trials
p_A = 0.3  # Click rate for variant A
p_B = 0.4  # Click rate for variant B

# Taqsimotlarni vizualizatsiya qilish
k = np.arange(0, n+1)
pmf_A = stats.binom.pmf(k, n, p_A)
pmf_B = stats.binom.pmf(k, n, p_B)

plt.figure(figsize=(15, 10))

# PMF comparison
plt.subplot(2, 2, 1)
plt.bar(k-0.2, pmf_A, width=0.4, alpha=0.7, label='Variant A (p=0.3)', color='red')
plt.bar(k+0.2, pmf_B, width=0.4, alpha=0.7, label='Variant B (p=0.4)', color='blue')
plt.title('A/B Test: Binomial taqsimotlar taqqoslash')
plt.xlabel('Kliklar soni')
plt.ylabel('Ehtimollik')
plt.legend()
plt.grid(True, alpha=0.3)

# CDF comparison
plt.subplot(2, 2, 2)
cdf_A = stats.binom.cdf(k, n, p_A)
cdf_B = stats.binom.cdf(k, n, p_B)
plt.step(k, cdf_A, where='post', linewidth=2, label='Variant A CDF', color='red')
plt.step(k, cdf_B, where='post', linewidth=2, label='Variant B CDF', color='blue')
plt.title('Kumulyativ taqsimot funksiyalari')
plt.xlabel('Kliklar soni')
plt.ylabel('P(X ≤ k)')
plt.legend()
plt.grid(True, alpha=0.3)

# 1000 marta simulatsiya
simulations = 1000
results_A = np.random.binomial(n, p_A, simulations)
results_B = np.random.binomial(n, p_B, simulations)

# Simulatsiya natijalari
plt.subplot(2, 2, 3)
plt.hist(results_A, bins=np.arange(-0.5, n+1.5, 1), alpha=0.7, label='Variant A', color='red', density=True)
plt.hist(results_B, bins=np.arange(-0.5, n+1.5, 1), alpha=0.7, label='Variant B', color='blue', density=True)
plt.title('Simulatsiya natijalari (1000 marta)')
plt.xlabel('Kliklar soni')
plt.ylabel('Nisbiy chastota')
plt.legend()
plt.grid(True, alpha=0.3)

# Win rate analysis
wins_B = np.sum(results_B > results_A)
ties = np.sum(results_B == results_A)
wins_A = simulations - wins_B - ties

plt.subplot(2, 2, 4)
categories = ['Variant A\nyutadi', 'Durrang', 'Variant B\nyutadi']
values = [wins_A, ties, wins_B]
colors = ['red', 'gray', 'blue']
plt.pie(values, labels=categories, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('1000 ta eksperiment natijasi')

plt.tight_layout()
plt.show()

# Statistik tahlil
mean_A, var_A = n * p_A, n * p_A * (1 - p_A)
mean_B, var_B = n * p_B, n * p_B * (1 - p_B)

print("Nazariy statistikalar:")
print(f"Variant A: O'rtacha = {mean_A:.1f}, Dispersiya = {var_A:.2f}")
print(f"Variant B: O'rtacha = {mean_B:.1f}, Dispersiya = {var_B:.2f}")

print(f"\nSimulatsiya statistikalari:")
print(f"Variant A: O'rtacha = {np.mean(results_A):.2f}, Dispersiya = {np.var(results_A):.2f}")
print(f"Variant B: O'rtacha = {np.mean(results_B):.2f}, Dispersiya = {np.var(results_B):.2f}")

# Normal approximation check (np > 5 va n(1-p) > 5)
print(f"\nNormal yaqinlashuv shartlari:")
print(f"Variant A: np = {n*p_A}, n(1-p) = {n*(1-p_A)} - {'✓' if n*p_A > 5 and n*(1-p_A) > 5 else '✗'}")
print(f"Variant B: np = {n*p_B}, n(1-p) = {n*(1-p_B)} - {'✓' if n*p_B > 5 and n*(1-p_B) > 5 else '✗'}")

# Effect size
effect_size = (mean_B - mean_A) / np.sqrt((var_A + var_B) / 2)
print(f"\nEffect size (Cohen's d): {effect_size:.3f}")

print(f"\nXulosa:")
print(f"Variant B {wins_B/simulations*100:.1f}% holatlarda yaxshiroq natija ko'rsatdi.")
print(f"Bu statistik jihatdan ahamiyatli farq ekanligini ko'rsatadi.")

## Mashq 3: Uniform taqsimot - Monte Carlo simulatsiyasi

**Vazifa:** $a=-5$, $b=5$ bo'lgan uniform taqsimotdan foydalanib quyidagilarni bajaring:

1. Monte Carlo usuli bilan π (pi) qiymatini hisoblang
2. Turli sample size'lar bilan aniqlikni taqqoslang
3. Uniform taqsimotdan normal taqsimotga o'tish (Central Limit Theorem)
4. Bootstrap usuli bilan confidence interval hisoblang

**Yechim:**

In [ ]:
# Monte Carlo simulatsiyasi π hisoblash uchun
def estimate_pi(n_samples):
    """Monte Carlo method to estimate π using uniform distribution"""
    # Generate random points in [-1, 1] x [-1, 1] square
    x = np.random.uniform(-1, 1, n_samples)
    y = np.random.uniform(-1, 1, n_samples)
    
    # Check how many points are inside unit circle
    inside_circle = (x**2 + y**2) <= 1
    pi_estimate = 4 * np.sum(inside_circle) / n_samples
    return pi_estimate

# Turli sample size'lar bilan π hisoblash
sample_sizes = [100, 1000, 10000, 100000, 1000000]
pi_estimates = []
errors = []

plt.figure(figsize=(15, 12))

# π estimation with different sample sizes
plt.subplot(2, 3, 1)
for n in sample_sizes:
    pi_est = estimate_pi(n)
    pi_estimates.append(pi_est)
    error = abs(pi_est - np.pi)
    errors.append(error)
    print(f"n = {n:7d}: π ≈ {pi_est:.6f}, xato = {error:.6f}")

plt.loglog(sample_sizes, errors, 'bo-', linewidth=2, markersize=8)
plt.title('Monte Carlo aniqlik vs Sample Size')
plt.xlabel('Sample Size')
plt.ylabel('Absolute Error')
plt.grid(True, alpha=0.3)

# Central Limit Theorem demonstration
plt.subplot(2, 3, 2)
a, b = -5, 5
sample_sizes_clt = [1, 5, 10, 30]
x_range = np.linspace(-6, 6, 100)

for i, n in enumerate(sample_sizes_clt):
    # Generate means of samples
    sample_means = []
    for _ in range(1000):
        sample = np.random.uniform(a, b, n)
        sample_means.append(np.mean(sample))
    
    plt.hist(sample_means, bins=30, alpha=0.6, density=True, 
             label=f'n={n}', color=plt.cm.viridis(i/len(sample_sizes_clt)))

# Theoretical normal distribution for large n
theoretical_mean = (a + b) / 2  # 0
theoretical_std = np.sqrt((b - a)**2 / 12) / np.sqrt(30)  # For n=30
plt.plot(x_range, stats.norm.pdf(x_range, theoretical_mean, theoretical_std), 
         'r-', linewidth=2, label='Nazariy Normal (n=30)')

plt.title('Central Limit Theorem: Uniform → Normal')
plt.xlabel('Sample Mean')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)

# Uniform distribution properties
plt.subplot(2, 3, 3)
data = np.random.uniform(a, b, 1000)
plt.hist(data, bins=30, density=True, alpha=0.7, color='lightgreen', edgecolor='black')
x_uniform = np.linspace(a-1, b+1, 100)
y_uniform = np.where((x_uniform >= a) & (x_uniform <= b), 1/(b-a), 0)
plt.plot(x_uniform, y_uniform, 'r-', linewidth=3, label='Nazariy zichlik')
plt.title('Uniform(-5, 5) taqsimot')
plt.xlabel('Qiymat')
plt.ylabel('Zichlik')
plt.legend()
plt.grid(True, alpha=0.3)

# Bootstrap confidence intervals
plt.subplot(2, 3, 4)
original_sample = np.random.uniform(a, b, 100)
bootstrap_means = []
n_bootstrap = 1000

for _ in range(n_bootstrap):
    bootstrap_sample = np.random.choice(original_sample, size=len(original_sample), replace=True)
    bootstrap_means.append(np.mean(bootstrap_sample))

bootstrap_means = np.array(bootstrap_means)
plt.hist(bootstrap_means, bins=30, alpha=0.7, color='salmon', edgecolor='black')
plt.axvline(np.mean(bootstrap_means), color='red', linestyle='--', linewidth=2, label='Bootstrap o\'rtacha')
plt.axvline(np.mean(original_sample), color='blue', linestyle='--', linewidth=2, label='Asl o\'rtacha')
plt.title('Bootstrap Sampling Distribution')
plt.xlabel('Bootstrap Means')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

# Monte Carlo π visualization
plt.subplot(2, 3, 5)
n_viz = 5000
x_viz = np.random.uniform(-1, 1, n_viz)
y_viz = np.random.uniform(-1, 1, n_viz)
inside = (x_viz**2 + y_viz**2) <= 1

plt.scatter(x_viz[inside], y_viz[inside], c='red', s=1, alpha=0.6, label='Aylana ichida')
plt.scatter(x_viz[~inside], y_viz[~inside], c='blue', s=1, alpha=0.6, label='Aylana tashqarida')

# Draw unit circle
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), 'black', linewidth=2)
plt.axis('equal')
plt.title('Monte Carlo π hisoblash')
plt.legend()
plt.grid(True, alpha=0.3)

# Convergence of π estimate
plt.subplot(2, 3, 6)
n_points = 10000
x_conv = np.random.uniform(-1, 1, n_points)
y_conv = np.random.uniform(-1, 1, n_points)
inside_conv = (x_conv**2 + y_conv**2) <= 1

pi_estimates_conv = []
for i in range(1, n_points, 100):
    pi_est = 4 * np.sum(inside_conv[:i]) / i
    pi_estimates_conv.append(pi_est)

plt.plot(range(1, n_points, 100), pi_estimates_conv, 'b-', alpha=0.7)
plt.axhline(y=np.pi, color='red', linestyle='--', linewidth=2, label='π = 3.14159...')
plt.title('π baholashning yaqinlashuvi')
plt.xlabel('Sample Size')
plt.ylabel('π estimate')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Bootstrap confidence intervals calculation
confidence_level = 0.95
alpha = 1 - confidence_level
lower_percentile = (alpha/2) * 100
upper_percentile = (1 - alpha/2) * 100

ci_lower = np.percentile(bootstrap_means, lower_percentile)
ci_upper = np.percentile(bootstrap_means, upper_percentile)

print(f"\nBootstrap Statistics:")
print(f"Asl sample o'rtachasi: {np.mean(original_sample):.4f}")
print(f"Bootstrap o'rtachasi: {np.mean(bootstrap_means):.4f}")
print(f"Bootstrap standart xatosi: {np.std(bootstrap_means):.4f}")
print(f"{confidence_level*100}% Bootstrap CI: [{ci_lower:.4f}, {ci_upper:.4f}]")

print(f"\nπ bahosining yakuniy natijasi:")
final_pi = estimate_pi(1000000)
print(f"1 million nuqta bilan π ≈ {final_pi:.6f}")
print(f"Haqiqiy π = {np.pi:.6f}")
print(f"Absolute error = {abs(final_pi - np.pi):.6f}")

## Mashq 4: Eksponensial taqsimot - Real vaqt tahlili

**Vazifa:** Call center'da qo'ng'iroqlar orasidagi vaqt $\lambda=2$ (daqiqada 2 ta qo'ng'iroq) bo'lgan eksponensial taqsimot bilan modellanadi.

Quyidagilarni bajaring:
1. 8 soatlik ish kunida qancha qo'ng'iroq kutiladi?
2. 5 daqiqadan ko'p kutish ehtimolligini hisoblang
3. Poisson jarayoni bilan bog'lanishni ko'rsating
4. Survival analysis (qo'ng'iroq kelmaslik ehtimolligi)

**Yechim:**

In [ ]:
# Call center parametrlari
lambda_rate = 2  # calls per minute
work_hours = 8  # hours
work_minutes = work_hours * 60  # minutes

# Eksponensial taqsimot parametri (kutish vaqti)
scale = 1 / lambda_rate  # mean waiting time in minutes

plt.figure(figsize=(16, 12))

# 1. Eksponensial taqsimot zichlik va survival funksiyasi
plt.subplot(3, 3, 1)
x = np.linspace(0, 10, 1000)
pdf = stats.expon.pdf(x, scale=scale)
cdf = stats.expon.cdf(x, scale=scale)
survival = 1 - cdf

plt.plot(x, pdf, 'b-', linewidth=2, label='PDF (Zichlik)')
plt.plot(x, survival, 'r-', linewidth=2, label='Survival Function')
plt.title('Eksponensial taqsimot funksiyalari')
plt.xlabel('Vaqt (daqiqa)')
plt.ylabel('Ehtimollik / Survival')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. 8 soatlik ish kuni simulatsiyasi
plt.subplot(3, 3, 2)
# Generate inter-arrival times
np.random.seed(42)
inter_arrival_times = np.random.exponential(scale, 1000)
arrival_times = np.cumsum(inter_arrival_times)
calls_in_workday = arrival_times[arrival_times <= work_minutes]

plt.hist(inter_arrival_times[:200], bins=30, density=True, alpha=0.7, color='lightblue')
x_theory = np.linspace(0, max(inter_arrival_times[:200]), 100)
plt.plot(x_theory, stats.expon.pdf(x_theory, scale=scale), 'r-', linewidth=2, label='Nazariy PDF')
plt.title('Qo\'ng\'iroqlar orasidagi vaqt')
plt.xlabel('Vaqt (daqiqa)')
plt.ylabel('Zichlik')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Qo'ng'iroqlar soni vaqt bo'yicha
plt.subplot(3, 3, 3)
time_points = np.arange(0, work_minutes, 10)
call_counts = [np.sum(calls_in_workday <= t) for t in time_points]
plt.plot(time_points/60, call_counts, 'b-', linewidth=2, label='Qo\'ng\'iroqlar soni')
plt.plot(time_points/60, lambda_rate * time_points, 'r--', linewidth=2, label='Kutilgan son')
plt.title('8 soatlik ish kunidagi qo\'ng\'iroqlar')
plt.xlabel('Vaqt (soat)')
plt.ylabel('Qo\'ng\'iroqlar soni')
plt.legend()
plt.grid(True, alpha=0.3)

# 4. Poisson-Exponential relationship
plt.subplot(3, 3, 4)
# Poisson distribution for number of calls in 1 minute intervals
minute_intervals = np.arange(1, 11)
poisson_probs = [stats.poisson.pmf(k, lambda_rate) for k in minute_intervals]
plt.bar(minute_intervals, poisson_probs, alpha=0.7, color='orange', label='Poisson(λ=2)')
plt.title('1 daqiqadagi qo\'ng\'iroqlar soni (Poisson)')
plt.xlabel('Qo\'ng\'iroqlar soni')
plt.ylabel('Ehtimollik')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Waiting time analysis
plt.subplot(3, 3, 5)
waiting_times = [1, 2, 3, 4, 5, 10]
probabilities = [1 - stats.expon.cdf(t, scale=scale) for t in waiting_times]
plt.bar(waiting_times, probabilities, alpha=0.7, color='salmon')
plt.title('Kutish vaqti ehtimolliklari')
plt.xlabel('Kutish vaqti (daqiqa)')
plt.ylabel('P(X > t)')
plt.grid(True, alpha=0.3)

# 6. Memoryless property demonstration
plt.subplot(3, 3, 6)
t_values = np.linspace(0, 10, 100)
conditional_survival = np.exp(-lambda_rate * t_values)  # P(X > t+s | X > s) = P(X > t)
plt.plot(t_values, conditional_survival, 'g-', linewidth=3, label='P(X > t+2 | X > 2)')
plt.plot(t_values, 1 - stats.expon.cdf(t_values, scale=scale), 'r--', linewidth=2, label='P(X > t)')
plt.title('Xotirasizsiz xususiyat')
plt.xlabel('Vaqt t')
plt.ylabel('Ehtimollik')
plt.legend()
plt.grid(True, alpha=0.3)

# 7. Hazard function (λ = constant for exponential)
plt.subplot(3, 3, 7)
hazard_rate = np.full_like(x, lambda_rate)
plt.plot(x, hazard_rate, 'purple', linewidth=3)
plt.title('Hazard Function (λ = 2)')
plt.xlabel('Vaqt (daqiqa)')
plt.ylabel('Hazard Rate')
plt.ylim(0, 3)
plt.grid(True, alpha=0.3)

# 8. Reliability analysis
plt.subplot(3, 3, 8)
reliability = np.exp(-lambda_rate * x)
plt.plot(x, reliability, 'brown', linewidth=3, label='Reliability R(t)')
plt.plot(x, 1 - reliability, 'navy', linewidth=2, label='Cumulative Failure F(t)')
plt.title('Reliability Analysis')
plt.xlabel('Vaqt (daqiqa)')
plt.ylabel('Ehtimollik')
plt.legend()
plt.grid(True, alpha=0.3)

# 9. Daily call distribution
plt.subplot(3, 3, 9)
daily_calls = []
for day in range(100):
    inter_arrivals = np.random.exponential(scale, 2000)
    arrivals = np.cumsum(inter_arrivals)
    daily_calls.append(len(arrivals[arrivals <= work_minutes]))

plt.hist(daily_calls, bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
expected_daily_calls = lambda_rate * work_minutes
plt.axvline(expected_daily_calls, color='red', linestyle='--', linewidth=2, 
           label=f'Kutilgan: {expected_daily_calls}')
plt.axvline(np.mean(daily_calls), color='blue', linestyle='--', linewidth=2, 
           label=f'Amaliy: {np.mean(daily_calls):.1f}')
plt.title('Kunlik qo\'ng\'iroqlar taqsimoti (100 kun)')
plt.xlabel('Qo\'ng\'iroqlar soni')
plt.ylabel('Chastota')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Detailed calculations and analysis
print("=== CALL CENTER TAHLILI ===")
print(f"Lambda rate: {lambda_rate} qo'ng'iroq/daqiqa")
print(f"O'rtacha kutish vaqti: {scale:.2f} daqiqa")

print(f"\n1. 8 soatlik ish kunida kutilgan qo'ng'iroqlar soni:")
expected_calls = lambda_rate * work_minutes
print(f"   Kutilgan: {expected_calls} qo'ng'iroq")
print(f"   Simulatsiya (100 kun o'rtachasi): {np.mean(daily_calls):.1f} qo'ng'iroq")

print(f"\n2. 5 daqiqadan ko'p kutish ehtimolligi:")
prob_wait_5min = 1 - stats.expon.cdf(5, scale=scale)
print(f"   P(X > 5) = {prob_wait_5min:.4f} = {prob_wait_5min*100:.2f}%")

print(f"\n3. Turli kutish vaqtlari uchun ehtimolliklar:")
for t in [1, 2, 3, 5, 10]:
    prob = 1 - stats.expon.cdf(t, scale=scale)
    print(f"   P(X > {t} daqiqa) = {prob:.4f} = {prob*100:.2f}%")

print(f"\n4. Xotirasizsiz xususiyat:")
print(f"   P(X > 3 | X > 1) = P(X > 2) = {np.exp(-lambda_rate * 2):.4f}")
print(f"   Bu eksponensial taqsimotning asosiy xususiyati!")

print(f"\n5. Poisson bog'lanish:")
print(f"   1 daqiqada kutilgan qo'ng'iroqlar: λ = {lambda_rate}")
print(f"   1 daqiqada 0 qo'ng'iroq ehtimolligi: {stats.poisson.pmf(0, lambda_rate):.4f}")
print(f"   1 daqiqada 3+ qo'ng'iroq ehtimolligi: {1 - stats.poisson.cdf(2, lambda_rate):.4f}")

## Xulosa - 2-guruh

2-guruh uchun chuqur amaliy mashqlar yakunlandi. Sizlar quyidagi professional ko'nikmalarni egallaidingiz:

### O'rganilgan mavzular:

**Normal taqsimot - Professional tahlil:**
- Z-score standartlashtirish
- Confidence intervals hisoblash
- Outlier detection (IQR va Z-score usullari)
- Q-Q plot orqali normallik tekshiruvi

**Binomial taqsimot - A/B Testing:**
- Real loyiha simulatsiyasi
- Statistical significance testing
- Effect size hisoblash
- Normal approximation tekshiruvi

**Uniform taqsimot - Monte Carlo usullari:**
- π qiymatini hisoblash
- Central Limit Theorem namoyishi
- Bootstrap confidence intervals
- Convergence analysis

**Eksponensial taqsimot - Real vaqt modellari:**
- Call center analysis
- Survival function va reliability
- Xotirasizsiz xususiyat
- Poisson jarayoni bilan bog'lanish

### Professional ko'nikmalar:
- Monte Carlo simulatsiyalar
- Bootstrap usullari
- Statistical hypothesis testing
- Real ma'lumotlar tahlili
- Professional vizualizatsiya

Bu ko'nikmalar data science va statistika sohasida ishlash uchun zarurdir!